<a href="https://colab.research.google.com/github/jamissi/Estudos-DataScience/blob/main/Big_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#***SENAI: BIG DATA***

Estrutura de repetição até ser verdade:
```
while true:
```

Estrutura de repetição:

```
For i in range():
```



Para utilizar vários argumentos dentro de uma função:

```
*args - para utilizar varios argumentos  / **kwargs - para chave e valor

```

#***PYSPARK***

In [3]:
#Criando ambiente SPARK e criando bibliotecas necessárias:
!pip install pyspark
!pip install findspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285388 sha256=d6d41ece5de78a652533fd56fce8e50f0914bacd765afb353b5e9abf81c26977
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [4]:
#Importando bibliotecas instaladas:
import findspark
from pyspark.sql import SparkSession

In [5]:
#Importando funções do pyspark:
from pyspark.sql import functions as f

In [6]:
#Criando conexão com o SPARK
spark = SparkSession.builder.appName("JAMI").getOrCreate() #criando um objeto

In [7]:
spark

---

In [9]:
#@title Criando um dataframe com o ***PYSPARK***
dados = [("Gol", 1997), ("Yaris", 2019), ("Monza", 1986), ("Maverick", 1970)]
dados

[('Gol', 1997), ('Yaris', 2019), ('Monza', 1986), ('Maverick', 1970)]

In [10]:
colunas = ["Modelo", "Ano"]

In [11]:
#visualizando os tipos de dados das colunas do df
df = spark.createDataFrame(dados, colunas)
df

DataFrame[Modelo: string, Ano: bigint]

In [ ]:
df.show(2, truncate=False) #para quebrar no meio os dados será necessário colocar true no truncate

In [ ]:
df.toPandas()

In [ ]:
#title buscando dados especificos. Para exibir a pesquisa é necessário utilizar o SHOW()
df.where("Ano <= 2000").toPandas()

In [ ]:
df.where("Ano <= 2000").show()

In [ ]:
#@title Criando dataframe com listas
dados = [
    ("João", 18, "M"),
    ("Maria", 20, "F"),
    ("José", 18, "M"),
    ("Alex", 46, "M")
         ]

colunas = ["Alunos", "Idade", "Sexo"]

#criando o dataframe
df2 = spark.createDataFrame(dados, colunas)

#visualizando o df criado
df2.show()

In [ ]:
df2.where("idade > 18").show()

In [ ]:
df2.where("Alunos like '%o%'").show()  #utilizando like para filtrar o dataframe

In [ ]:
#@title Importando arquivo .csv para o ambiente spark
empresa = spark.read.options(header = True).csv(
    "/content/drive/MyDrive/Python/empresas/part*",
    sep= ";",
    inferSchema=True
)
#caminho
#empresa.show()
#empresa.orderBy(cols("razao_social".asc())).show(50, False)

In [ ]:
#@title Visualizando esquema do DataFrame
empresa.printSchema()

root
 |-- cnpj: integer (nullable = true)
 |-- razao_social: string (nullable = true)
 |-- natureza_juridica: integer (nullable = true)
 |-- qualificacao_responsavel: integer (nullable = true)
 |-- capital_social: string (nullable = true)
 |-- porte: integer (nullable = true)
 |-- ente_federativo: string (nullable = true)



In [ ]:
#@title Verificando quantidade de registros(dados, linhas)
empresa.count()
print(f'{empresa.count():,.2f}')

4,585,679.00


******


##FAZENDO UM DATAFRAME

In [ ]:

empresa\
    .where("razao_social like '%MERCADAO%'")\
    .limit(7).show()

In [ ]:
empresa\
    .where("razao_social like '%MERCADO%'")\
    .count()

15116

In [ ]:
empresa\
    .where("razao_social like '%MERCADO%'")\
    .orderBy("razao_social")\
    .show()

In [ ]:
empresa\
  .where("razao_social like '%HOSPITAL%'")\
  .orderBy("razao_social")\
  .show(10, False)

In [ ]:
empresa\
    .where("razao_social like '%HOSPITAL%'")\
    .count()

##ALTERANDO COLUNAS DO DATAFRAME

--> TROCANDO O NOME DAS COLUNAS
--> TROCANDO CARACTERES POR OUTROS
--> MUDANDO O TIPO DE DADO


In [ ]:
empresa = empresa.withColumnRenamed("razao_social", "empresa")
empresa = empresa.withColumnRenamed("qualificacao_responsavel", "qualificacao")

In [ ]:
empresa.show()

In [ ]:
#@title Alterando campos no DataFrame com função REGEX:
empresa = empresa.withColumn("capital_social", f.regexp_replace("capital_social", ",", "."))
empresa.show()

+-----+--------------------+-----------------+------------+--------------+-----+---------------+
| cnpj|             empresa|natureza_juridica|qualificacao|capital_social|porte|ente_federativo|
+-----+--------------------+-----------------+------------+--------------+-----+---------------+
|  306|FRANCAMAR REFRIGE...|             2240|          49|          0.00|    1|           null|
| 1355|BRASILEIRO & OLIV...|             2062|          49|          0.00|    5|           null|
| 4820|REGISTRO DE IMOVE...|             3034|          32|          0.00|    5|           null|
| 5347|ROSELY APARECIDA ...|             2135|          50|          0.00|    5|           null|
| 6846|BADU E FILHOS TEC...|             2062|          49|       4000.00|    1|           null|
| 8416|  ELETRICA RUBI LTDA|             2062|          49|          0.00|    5|           null|
| 8992|SHIROMA VEICULOS ...|             2062|          49|          0.00|    5|           null|
| 9091|CONTATOS BAR E LA...|  

In [ ]:
#@title Convertando o tipo de dados das colunas utilizando cast():
empresa = empresa.withColumn("capital_social", empresa.capital_social.cast("float"))

In [ ]:
empresa.show()

In [ ]:
#@title Análise Exploratória, verificando quantas empresas possuem capitais = 0:
empresa\
  .where("capital_social = 0")\
  .count()
  #.show()

1691023

In [ ]:
#@title Análise Exploratória, verificando quantas empresas possuem capitais != 0:
empresa\
  .where("capital_social != 0")\
  .count()

In [ ]:
empresa2 = empresa\
  .where("capital_social != 0")

empresa2.show()

In [ ]:
empresa2\
  .where("capital_social >= 1000000")\
  .orderBy("capital_social", ascending=False)\
  .limit(15).toPandas()

### ***SQL NO DATAFRAME***

In [ ]:
#@title Criando uma view para utilização no spark.sql
empresa.createOrReplaceTempView("v_empresa")

In [ ]:
spark.sql("""
SELECT PORTE, EMPRESA
FROM V_EMPRESA
WHERE
  EMPRESA LIKE '%RESTAURANTE%'
          """).show(20, Fals)

#***PYTHON***

##OO

Entendendo a Orientação a Objetos:



```
Class, def __init__(self)
```



In [ ]:
#@title Entendendo a oritentação a objetos:
#definindo a classe
class Pessoa:
  #iniciando o metodo construtor do objeto
    def __init__(self, nome, idade, altura):
        self.nome = nome
        self.idade = idade
        self.altura = altura

    def andar(self):
      print(f"{self.nome} está andando...")

    def falar(self, frase):
      print(f'{self.nome} diz: - {frase}')

    def dormir(self):
      print(f'{self.nome} está dormindo, zzzZZz')

    def frio(self):
      print(f'{self.nome} está com frio!')

    def niver(self):
      print(f'{self.nome} está fazendo aniversário ela tem {self.idade} e fará {self.idade + 1}')


p1 = Pessoa("Janice", 28, 1.74)
p1.frio()
p1.niver()
p1.falar("FOME!")

Herança na orientação a objetos

```
class Alunos(Pessoa):
    def __init__(self, nome, idade, altura, celular, email):
      super().__init__(nome, idade, altura)
      self.celular = celular
      self.email = email

```
A classe a aluno está herdando os atributos da classe Pessoa.

Utilizamos o método ```super()``` para trazer os atributos herdados da classe Pessoa.

In [ ]:
#@title Herança
#from OOSenai.Pessoa import Pessoa

class Professor(Pessoa): # <-- herdando da classe pessoas
    def __init__(self, nome, idade, altura, area):
        super().__init__(nome, idade, altura) #<-- o atributos da classe pessoas
        self.area = area

    def apresentarProfessor(self):
        print(f'Nome: {self.nome}\n'
              f'Idade: {self.idade}\n'
              f'Area: {self.area}')



---

### ***Estrutura de repetição***

```
FOR I IN ...():  |  WHILE:
```


In [ ]:
#@title 1) Crie uma lista manualmente com 5 elementos. Mostre todos os valores da lista usando o comando FOR:

total = 0
lista = [
    [1,"Motorola", 3000, 4],
    [2,"Iphone", 7000, 2],
    [3,"Samsung", 5000, 3],
]
for i in lista:
  print(f'Produto: {i[1]} | Preço: {i[2]} | Qtd: {i[3]} | Total: {i[2] * i[3]}')
  total += i[2]* i[3]
print(f'TOTAL: R${total:.2F}')


Produto: Motorola | Preço: 3000 | Qtd: 4 | Total: 12000
Produto: Iphone | Preço: 7000 | Qtd: 2 | Total: 14000
Produto: Samsung | Preço: 5000 | Qtd: 3 | Total: 15000
TOTAL: R$41000.00


In [ ]:
#@title 2) Crie um programa que receba 2 valores p/ cáculo de uma exponenciação. O primeiro valor será a base e o segundo valor a quantidade de repetições. Para cada repetição a seguinte fórmula será executada: x = x + base^2. Mostre, cada repetição, o valor de x. A repetição deve ser feita usando o comando while.
base = int(input('Digite um valor: '))
qtd = int(input('Digite quantas vezes será repetido: '))
x = 0
cont = 0
print(f'x + {base}² = {x}')
while cont < qtd:
  x += b**2
  print(f'x + {base}² = {x}')
  cont += 1



Digite um valor: 3
Digite quantas vezes será repetido: 4
0 + 3² = 0
9 + 3² = 9
18 + 3² = 18
27 + 3² = 27
36 + 3² = 36


\

##Laço for

In [ ]:
#@title Utilizando FOR:
alunos = ['Isabel', 'Helena', 'Lucas', 'Gabriel']
print('Alunos Cadastrados')
#usando for
for aluno in alunos:
  print(f'Aluno: {aluno}')

In [ ]:
#@title FOR: mostrando itens dentro da lista
#Lista dentro de lista
alunos = [
    ["Isabel", 25, "isa@gmail.com"],
    ["Helena", 32, "helena@gmail.com"],
    ["Lucas", 25, "lucas@gmail.com"],
    ["Gabriel", 25, "gab@gmail.com"]
]
for i in alunos: #passamos os indices da lista dentro da variavel do FOR = i
  print(f"Aluno: {i[0]:10}| Idade: {i[1]}| Email: {i[2]:15}")

In [ ]:
#@title FOR: Dicionário

##Laço While

In [ ]:
#@title WHILE:
cont = 0
while(cont < 7):
  print(f"Repetição {cont+1}")
  cont += 1
print('Fim da Repetição')


In [ ]:
#@title Conversão de temperatura com WHILE:
repetir = True
print('Bem-vindo ao programa de conversão de temperatura!')
while repetir == True:
  k = float(input('Digite a temperatura em kelvin: '))
  c = k - 273.15
  print(f'Kelvin = {k:.2f} | Celsius = {c:.2f}\n')
  cont = input('Deseja continuar? [S/N] ')
  if (cont.lower() != "s"):
    repetir = False
print('Programa finalizado!')

---

##Criando Funções

### ***Criando Funções:*** Para utilizar a função em outro arquivo é necessário importa-la



```
def nome_da_função(parametros)
```



In [ ]:
#@title FUNÇÕES:

def multiploDe5(num):
  if(num != 0):
    if(num % 5 == 0):
      return True
    else:
      return False
  else:
    return False

#UTILIZANDO A FUNÇÃO CRIADA

num = int(input("Digite um valor: "))
if multiploDe5(num):
  print(f'{num} é múltiplo de 5')
else:
  print(f'{num} não é múltiplo de 5')

In [ ]:
#@title FUNÇÕES PASSANDO DOIS PARÂMETROS:
def multiploDeX(num,x):
  if(num != 0):
    if(num % x == 0):
      return True
    else:
      return False
  else:
    return False

num = int(input("Digite um valor: "))
x = int(input("Digite um número p/ saber se é múltiplo: "))
if multiploDe5(num):
  print(f'{num} é múltiplo de {x}')
else:
  print(f'{num} não é múltiplo de {x}')

In [ ]:
#@title 1) Cria uma função que receba dois parâmetros: mensagem e quantidade. O nome da função será imprimaMuitos(  ). A função imprimirá a mensagem passada por parâmetro quantas vezes foi informado no parâmetro quantidade.

#COM LAÇO WHILE
def imprimaMuitos(msg, qtd):
  cont = qtd
  while cont > 0:
    print(msg)
    cont -= 1
imprimaMuitos("OLI", 12)

#COM O LAÇO FOR
def imprimaMuitosFor(ms, qt):
  for i in range(qt):
    print(ms)
imprimaMuitosFor("OI", 10)


In [ ]:
#@title 2) Crie uma função que passado o valor de pelo menos 1 parâmetro retorne um valor booleano. Use essa função em uma estrutura de decisão.

def ehPar(num):
  if num % 2 == 0:
    return True

num = int(input('Digite um número: '))
if ehPar(num):
  print(f'O nº {num} é par!')
else:
  print(f'O nº {num} não é par!')


##Funções ARGS

### **Função `ARGS`**

In [ ]:
def imprimaMuitos(*valores):
  for i in valores:
    print(i)

def ehParImpar(*inteiros):
  contp = 0
  conti = 0
  for i in inteiros:
    if (i % 2 == 0):
      contp += 1
    else:
      conti += 1
  print(f"Qtd de numeros pares: {contp}")
  print(f'Qtd de numeros impares: {conti}')

imprimaMuitos(24,2131,234,245)

ehParImpar(8,2,1,5,64,32,356,545,67)


24
2131
234
245
Qtd de numeros pares: 5
Qtd de numeros impares: 4


In [ ]:
#@title Exercícios com *ARGS (VÁRIOS ARGUMENTOS DENTRO DO PARÂMETRO)
def calcularMediaFor(*notas):
  soma = 0
  for n in notas:
    soma += n
  media = soma / len(notas)
  print(f"Media: {media:.2f}")
calcularMediaFor(9,2,8,7)


def calcularMedia(*notas):
  media = sum(notas) / len(notas)
  print(f"Média: {media:.2f}")
calcularMedia(2,3,4,5,6)


6.5
4.0


---

##HERANÇAS

In [ ]:
#@title Orientação a Objetos: Heranças

class Clinica:
    def __init__(self, medico, especializacao):
        self.medico = medico
        self.especializacao = especializacao

    def idade(self, idade):
        print(f'{self.medico} tem {idade} anos.')

    def especializa(self):
        print(f'{self.medico} é especializado em {self.especializacao}')

#from OO.Pai import Clinica

class Paciente(Clinica):
    def __init__(self, medico, especilizacao, nome):
        super().__init__(medico,especilizacao)
        self.nome = nome

    def consulta(self):
        print(f'{self.nome} tem consulta com {self.medico}|{self.especializacao}')


#from OO.Pai import Clinica

class Farmacia(Clinica):
    def __init__(self, medico, especializacao, nome, remedio):
        super().__init__(medico, especializacao)
        self.nome = nome
        self.remedio = remedio

    def pegaRemedio(self, quantidade):
        print(f"Paciente: {self.nome} | Dr(a).{self.medico} - {self.especializacao}\n"
              f"Remedio prescrito: {self.remedio} | {quantidade} caixas")



---

### ***Listas/Dicionário***

---

1) Crie uma lista manual em Python contendo três valores string que você desejar. Mostre na tela quantos elementos a lista tem. Adicione mais dois elementos através da função **input()** usando o método **append()**.
Mostre novamente quantos elementos tem na lista.





In [ ]:
#@title Resposta 1:
lista = ['Livro', 25, 'Lápis']
print(f'LISTA: {lista}, possui {len(lista)} elementos.\n')
cont = 0
while cont < 2:
    add_elementos = input('Adicione um elemento a lista: ')
    lista.append(add_elementos)
    cont += 1
print(f'lista: {lista}, possui agora {len(lista)} elementos.')

LISTA: ['Livro', 25, 'Lápis'], possui 3 elementos.

Adicione um elemento a lista: luz
Adicione um elemento a lista: iluminação
lista: ['Livro', 25, 'Lápis', 'luz', 'iluminação'], possui agora 5 elementos.


---

2) Crie um dicionário de dados em Python com os seguintes valores:

"Nome do produto", quantidade e valor unitário.
O nome do produto será a chave e a quantidade e valor serão uma lista.
Inserir dois produtos no dicionário através de dados vindos da função **input()**. *Fazer ***SEM*** estrutura de repetição*





In [ ]:
#@title Resposta 2:
dic = {}
produto1 = input('Digite o nome do 1º produto: ')
quant1 = int(input(f'Digite o quantidade do produto, {produto1}: '))
valor1 = float(input(f'Digite o valor do produto, {produto1}: R$'))

produto2 = input('\nDigite o nome do 2º produto: ')
quant2 = int(input(f'Digite o quantidade do produto, {produto2}: '))
valor2 = float(input(f'Digite o valor do produto, {produto2}: R$'))

#UTILIZANDO UPDATE
dic.update({produto1:[quant1, valor1]})
dic.update({produto2:[quant2, valor2]})

#UTILIZANDO VARIAVEL
'''dic[produto1] = [quant1, valor1]
dic[produto2] = [quant2 , valor2]'''

print('-'*100)
print('Visualizando produtos cadastrados:')

print(f'\nProduto \"{produto1}\" cadastrado. Quantidade {dic[produto1][0]}, Preço: R${dic[produto1][1]}')
print(f'Produto \"{produto2}\" cadastrado. Quantidade {dic[produto2][0]}, Preço: R${dic[produto2][1]}')

---

2.1) Com estrutura de repetição:



In [ ]:
#@title Resposta 2.1:
dic1 = {}

for i in range(1, 3):
  name_prod = input(f'Insira o nome do {i}º produto: ')
  quant_prod = input(f'Informe a quant do produto {name_prod}: ')
  price_prod = float(input(f'Preço a do produto {name_prod}: R$'))

  dic1.update({name_prod:[quant_prod, price_prod]})

  #dic1[name_prod] = [quant_prod, price_prod]

print('\nExibindo produtos cadastrados com sucesso...\n')

for name_prod, dados in dic1.items():
  quant_prod = dados[0]
  price_prod = dados[1]
  print(f'{name_prod}  → {quant_prod:}|R${price_prod}')

#01/07/2023 - ÚLTIMA AULA PYSPARK

In [ ]:
#@title Importando arquivo Estabelecimento:

estab = spark.read.options(header=True).csv(
    "/content/drive/MyDrive/Python/estabelecimentos/part*",
    sep=";",
    inferSchema = True
)

In [ ]:
estab.show()

In [ ]:
#@title Convertando coluna data_de_inicio_atividade para data:
estab = estab.withColumn("data_de_inicio_atividade", f.to_date("data_de_inicio_atividade", "yyyyMMdd"))

In [ ]:
estab.show(10,False)

In [ ]:
empresa.show()

### Criando Views para realizar o JOINS com Estabelecimentos e Empresas.

In [ ]:
#@title Criando uma view com o DF de **estabelecimentos e empresa** para uso com comando SQL:
estab.createOrReplaceTempView("v_estab")
empresa.createOrReplaceTempView("v_empresas")

In [ ]:
estab.count()

4836219

In [ ]:
spark.sql(
"""
SELECT *
FROM V_EMPRESAS AS E
INNER JOIN V_ESTAB AS ESTAB ON ESTAB.CNPJ_BASICO = E.CNPJ
ORDER BY E.CNPJ
"""
).show(100, False)

In [ ]:
#@title Criando um DATAFRAME com as informações do JOIN das duas views:
emp_estab = spark.sql(
    """
    SELECT * FROM
    V_EMPRESAS AS E
    INNER JOIN
    V_ESTAB AS ESTAB
    ON ESTAB.CNPJ_BASICO = E.CNPJ
    """
)

In [ ]:
type(emp_estab)

pyspark.sql.dataframe.DataFrame

###TRANSFORMANDO EMP_ESTAB EM ARQUIVO TIPO ORC 👹

In [ ]:
emp_estab.coalesce(1).write.orc(
  "/content/drive/MyDrive/Python/Saidas_Orc",
  mode = "overwrite"
)

In [ ]:
emp_estab = spark.read.orc(
    "/content/drive/MyDrive/Python/Saidas_Orc/part*"
)

In [ ]:
emp_estab.show(10, False)

+----+---------------------------------------------------+-----------------+------------+--------------+-----+---------------+-----------+----------+-------+---------------------------+------------------------+------------------+-----------------------+-------------------------+--------------------------+----+------------------------+---------------------+-----------------------+------------------+-----------------------+------+---------------------------------+-----------------+--------+---+---------+-----+----------+-----+----------+----------+-------+-------------------------+-----------------+-------------------------+
|cnpj|empresa                                            |natureza_juridica|qualificacao|capital_social|porte|ente_federativo|cnpj_basico|cnpj_ordem|cnpj_dv|identificador_matriz_filial|nome_fantasia           |situacao_cadastral|data_situacao_cadastral|motivo_situacao_cadastral|nome_da_cidade_no_exterior|pais|data_de_inicio_atividade|cnae_fiscal_principal|cnae_fiscal

In [ ]:
emp_estab.createOrReplaceTempView("v_emp_estab")

In [ ]:
spark.sql(
"""
SELECT * FROM
V_EMP_ESTAB
WHERE
CAPITAL_SOCIAL >= 1000000 AND CAPITAL_SOCIAL <= 5000000
ORDER BY
CNPJ
"""
).show(10, False)

+----+---------------------------------------------+-----------------+------------+--------------+-----+---------------+-----------+----------+-------+---------------------------+-----------------------------------+------------------+-----------------------+-------------------------+--------------------------+----+------------------------+---------------------+-------------------------------------------------------------------------------+------------------+------------------+------+---------------+-----------------+--------+---+---------+-----+----------+-----+----------+----------+--------+------------------------------------+-----------------+-------------------------+
|cnpj|empresa                                      |natureza_juridica|qualificacao|capital_social|porte|ente_federativo|cnpj_basico|cnpj_ordem|cnpj_dv|identificador_matriz_filial|nome_fantasia                      |situacao_cadastral|data_situacao_cadastral|motivo_situacao_cadastral|nome_da_cidade_no_exterior|pais|dat

In [ ]:
#@title Usando funções de agregação traga em tela a info de quantidade de estabelecimentos por estados.

spark.sql(
    """
    SELECT UF, COUNT(*) AS QTD_ESTABELECIMENTOS
    FROM V_EMP_ESTAB
    GROUP BY UF
    ORDER BY QTD_ESTABELECIMENTOS DESC
    """
).limit(50).toPandas()

,UF,QTD_ESTABELECIMENTOS
0,SP,1385303
1,MG,530016
2,RJ,416661
3,RS,352513
4,PR,323483
5,BA,246650
6,SC,216457
7,GO,166082
8,PE,148247
9,CE,137705


In [ ]:
!apt-get -qq install -y git
!git clone https://github.com/jamissi/bigdata_senai.git


Cloning into 'bigdata_senai'...
